In [1]:
from clingo.symbol import Number, String
from clingo.control import Control

In [2]:
def run(axioms):
    evASP = Control("0")
    evASP.add("base", [], axioms)

    evASP.ground([("base", [])])
    
    with evASP.solve(yield_=True) as handle:
        #print(handle)
        for m in handle: 
            print("Answer:\n{}\n".format(m))

In [3]:
code = """

causes(move, bat(B-1), bat(B)) :- bat(B), bat(B-1).
causes(move, pos(P+1), pos(P)) :- pos(P), pos(P+1).

pos(1..3).
bat(0..1).



"""

run(code)

Answer:
bat(0) bat(1) causes(move,pos(2),pos(1)) causes(move,pos(3),pos(2)) causes(move,bat(0),bat(1)) pos(1) pos(2) pos(3)



In [4]:
sokrat = """

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

causes(chargedWith(X,Y), inPrison(X), 1) :- person(X), mistake(Y).

causes(escapes(X, prison), -inPrison(X), 2) :- person(X).
cond(2, inPrison(X)) :- person(X).

causes(drinks(X,hemlock), -isAlive(X), 3) :- person(X).
cond(3, isAlive(X)) :- person(X).

impos(sentencedTo(X,Y), 4) :- person(X), sanction(Y).
cond(4, -inPrison(X)) :- person(X).

impos(walksOutside(X), 5) :- person(X).
cond(5, inPrison(X)) :- person(X).

mistake(impety).
sanction(death).
person(sokrates).

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#show holds/2.
#show causes/3.

"""

#################
meta = """
time(1..5).
holds(Head, T+1) :- causes(Act, Head, Nr), not condNotHolds(Nr, T), time(T).
condNotHolds(Nr, T) :- cond(Nr, Cond), not holds(Cond, T), time(T).

%holds(F,T+1) :- holds(F,T), not holds(-F,T+1), time(T).
%holds(-F,T+1) :- holds(-F,T), not holds(F, T+1).

% :- holds(X, T), holds(-X,T).

#show holds/2.
"""
#################


run(sokrat)

Answer:
causes(drinks(sokrates,hemlock),-isAlive(sokrates),3) causes(escapes(sokrates,prison),-inPrison(sokrates),2) causes(chargedWith(sokrates,impety),inPrison(sokrates),1)



In [5]:
run("""

fluent(get(t)).

holds(get(t)).
holds(-get(t)).

is :- holds(X), fluent(X).
no :- holds(X), holds(-X).

""")

Answer:
fluent(get(t)) holds(get(t)) holds(-get(t)) is no



In [6]:
AD = """

impo(e1, 5).
cond(5, a).

caus(e1, e, 6).
cond(6, -e).

caus(e2, d, 7).
cond(7, -d).

caus(e3, a, 8).
cond(8, -a).

caus(e3, c, 9).
cond(9, -c).

impo(e3, 10).
cond(10, -e).

impo(e3, 11).
cond(11, -f).

if(b,12).
cond(12, c).

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

time(1..4).


"""
run(AD)

Answer:
time(1) time(2) time(3) time(4) impo(e1,5) impo(e3,10) impo(e3,11) cond(5,a) cond(12,c) cond(6,-e) cond(7,-d) cond(8,-a) cond(9,-c) cond(10,-e) cond(11,-f) caus(e1,e,6) caus(e2,d,7) caus(e3,a,8) caus(e3,c,9) if(b,12)



In [7]:
scena = """

holds(-a,1; -b,1; -c,1; -d,1; -e,1; f,1).
occ(e1, 1).

holds(-a,2;-b,2;-c,2;-d,2;e,2;f,2).
occ(e2,2).

holds(-a,3;-b,3;-c,3;d,3;e,3;f,3). 
occ(e3,3). 

holds(a,4;b,4;c,4;d,4;e,4;f,4).


"""
run(scena)

Answer:
occ(e1,1) occ(e2,2) occ(e3,3) holds(-a,1) holds(-b,1) holds(-c,1) holds(-d,1) holds(-e,1) holds(f,1) holds(-a,2) holds(-b,2) holds(-c,2) holds(-d,2) holds(e,2) holds(f,2) holds(-a,3) holds(-b,3) holds(-c,3) holds(d,3) holds(e,3) holds(f,3) holds(a,4) holds(b,4) holds(c,4) holds(d,4) holds(e,4) holds(f,4)



In [8]:
theta = """
time(1..4).

% outcome
theta(a;b;c;d;e;f).

thetaNotSat(T) :- theta(X), not holds(X,T), time(T).
thetaSat(T) :- not thetaNotSat(T), time(T).

1{transit_ev(Ev,T)}1 :- occ(Ev, T), thetaNotSat(T), thetaSat(T+1).

"""

outcome_transit = scena+theta
run(outcome_transit)

Answer:
time(1) time(2) time(3) time(4) theta(a) theta(b) theta(c) theta(d) theta(e) theta(f) holds(-a,1) holds(-b,1) holds(-c,1) holds(-d,1) holds(-e,1) holds(f,1) holds(-a,2) holds(-b,2) holds(-c,2) holds(-d,2) holds(e,2) holds(f,2) holds(-a,3) holds(-b,3) holds(-c,3) holds(d,3) holds(e,3) holds(f,3) holds(a,4) holds(b,4) holds(c,4) holds(d,4) holds(e,4) holds(f,4) thetaNotSat(1) thetaNotSat(2) thetaNotSat(3) thetaSat(4) transit_ev(e3,3) occ(e1,1) occ(e2,2) occ(e3,3)



In [9]:
defs = """
time(1..4).
id(5..12).

gen_dir(Ev, T, Eff) :- occ(Ev, T), transit_ev(Ev,T), caus(Ev, Eff, Id), condHold(Id,T), id(Id).

condDontHold(Id, T) :- cond(Id, Cond), not holds(Cond, T), time(T), id(Id).
condHold(Id,T) :- not condDontHold(Id, T), time(T), id(Id).

out_dir_eff(Ev, T, Eff) :- theta(Eff), gen_dir(Ev, T, Eff).

%#show theta/1.
%#show gen_dir/3.
%#show out_dir_eff/3.


directly_caused(Ev, T, Eff) :- occ(Ev, T), caus(Ev, Eff, Id), condHold(Id,T), id(Id).
directly_caused(Ev, T, Eff) :- occ(Ev, T), holds(Eff, T), holds(Eff,T+1).
ind_eff(Ev, T, Eff) :- occ(Ev,T), holds(Eff,T+1), not directly_caused(Ev, T, Eff), time(T).

out_ind_eff(Ev, T, Eff) :- theta(Eff), ind_eff(Ev, T, Eff), transit_ev(Ev,T).

%#show out_ind_eff/3.

"""
basis = outcome_transit+defs+AD
run(basis)

Answer:
time(1) time(2) time(3) time(4) theta(a) theta(b) theta(c) theta(d) theta(e) theta(f) holds(-a,1) holds(-b,1) holds(-c,1) holds(-d,1) holds(-e,1) holds(f,1) holds(-a,2) holds(-b,2) holds(-c,2) holds(-d,2) holds(e,2) holds(f,2) holds(-a,3) holds(-b,3) holds(-c,3) holds(d,3) holds(e,3) holds(f,3) holds(a,4) holds(b,4) holds(c,4) holds(d,4) holds(e,4) holds(f,4) thetaNotSat(1) thetaNotSat(2) thetaNotSat(3) thetaSat(4) transit_ev(e3,3) occ(e1,1) occ(e2,2) occ(e3,3) id(5) id(6) id(7) id(8) id(9) id(10) id(11) id(12) condHold(6,1) condHold(7,1) condHold(8,1) condHold(9,1) condHold(10,1) condHold(7,2) condHold(8,2) condHold(9,2) condHold(8,3) condHold(9,3) condHold(5,4) condHold(12,4) caus(e1,e,6) caus(e2,d,7) caus(e3,a,8) caus(e3,c,9) gen_dir(e3,3,a) gen_dir(e3,3,c) cond(5,a) cond(12,c) cond(6,-e) cond(7,-d) cond(8,-a) cond(9,-c) cond(10,-e) cond(11,-f) condDontHold(5,1) condDontHold(5,2) condDontHold(5,3) condDontHold(12,1) condDontHold(12,2) condDontHold(12,3) condDontHold(6,2) con

### 1st Causal Exp

In [10]:
first = """


#show out_dir_eff/3.
#show out_ind_eff/3.

"""
run(first+basis)

Answer:
out_dir_eff(e3,3,a) out_dir_eff(e3,3,c) out_ind_eff(e3,3,b)



### 2nd Causal Exp

In [21]:
ensur_ev = """

l_transit_ev(Lit, Ev, T) :- occ(Ev,T), holds(Lit,T+1), not holds(Lit, T), time(T).

1{step(X): time(X)}1.
l_ensur_ev(Lit, Ev, T, T1) :- l_transit_ev(Lit, Ev, T1), T1 < T, time(T), step(T1).

:~ step(X). [1@1, X, step]

#show l_transit_ev/3.
#show step/1.
"""
run(ensur_ev+basis)

Answer:
l_transit_ev(e,e1,1) l_transit_ev(d,e2,2) l_transit_ev(a,e3,3) l_transit_ev(b,e3,3) l_transit_ev(c,e3,3) step(3)



In [33]:
outcome_sup = """
ev(Ev) :- occ(Ev,T), time(T).
dir_ind_eff(Ev,T,Eff) :- out_dir_eff(Ev,T,Eff).
dir_ind_eff(Ev,T,Eff) :- out_ind_eff(Ev,T,Eff).
remain_lit(Ev,T,Eff) :- theta(Eff), occ(EV,T), transit_ev(Ev,T), not dir_ind_eff(Ev,T,Eff), time(T), ev(Ev).

%#show dir_ind_eff/3.
#show remain_lit/3.
"""

run(outcome_sup+basis)

Answer:
remain_lit(e3,3,d) remain_lit(e3,3,e) remain_lit(e3,3,f)



### 3rd Causal Exp

In [43]:
prec = """

prec(Ev, Eff) :- caus(Ev,H,Id), cond(Id, Eff).
prec(Ev, Eff) :- impo(Ev,Id), cond(Id, -Eff).

#show prec/2.

"""

run(prec+AD)

Answer:
prec(e1,-a) prec(e3,e) prec(e3,f) prec(e1,-e) prec(e2,-d) prec(e3,-a) prec(e3,-c)



In [ ]:
transit_sup = """



"""

In [ ]:
run("""


""")

In [11]:
run("""
{a;b;c}1.

""")

Answer:


Answer:
a

Answer:
b

Answer:
c

